## Computation Graphs 

When we design software to implement neural networks, we want to come up with a way that can allow us to seamlessly compute the gradients, regardless of the architecture type so that the programmer doesn't have to manually compute gradients when changes are made to the network. 

We galvanise this idea in form of a data structure called a **Computation graph**. A computation graph looks very similar to the diagram of the graph that we made in the image above. However, the nodes in a computation graph are basically operators. These operators are basically the mathematical operators except for one case, where we need to represent creation of a user-defined variable.

![Computation Graph](images/computation_graph_forward.png)

Notice that we have also denoted the leaf variables $ a, w_1, w_2, w_3, w_4 $ in the graph for sake of clarity. However, it should noted that they are not a part of the computation graph. What they represent in our graph is the special case for user-defined variables which we just covered as an exception.

The variables, b,c and d are created as a result of mathematical operations, whereas variables a, w1, w2, w3 and w4 are initialised by the user itself. Since, they are not created by any mathematical operator, nodes corresponding to their creation is represented by their name itself. This is true for all the leaf nodes in the graph. 

## Tensors: Basic Building Blocks of PyTorch

Tensor is a data structure which is a fundamental building block of PyTorch. Tensors are pretty much like numpy arrays, except that unlike numpy, tensors are designed to take advantage of parallel computation capabilities of a GPU. A lot of Tensor syntax is similar to that of numpy arrays. 

In [5]:
import torch

tsr = torch.Tensor(3,5)

print(tsr)


tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0785e-30],
        [1.4013e-45, 3.2751e-35, 1.4013e-45, 1.0640e-30, 1.4013e-45],
        [3.1121e-35, 1.4013e-45, 1.0996e-30, 1.4013e-45, 4.4889e-29]])


One it's own, Tensor is just like a numpy ndarray. A data structure that can let you do fast linear algebra options. If you want PyTorch to create a graph corresponding to these operations, you will have to set the `requires_grad` attribute of the Tensor to True. 

The API can be a bit confusing here. There are multiple ways to initialise tensors in PyTorch. While some ways can let you explicitly define that the requires_grad in the constructor itself, others require you to set it manually after creation of the Tensor.

In [6]:
t1 = torch.randn((3,3), requires_grad = True) 

t2 = torch.FloatTensor(3,3) # No way to specify requires_grad while initiating 
t2.requires_grad = True

`requires_grad` is contagious. It means that when a Tensor is created by operating on other Tensors, the requires_grad of the resultant Tensor would be set True given at least one of the tensors used for creation has it's `requires_grad` set to `True`.

Each Tensor has a something an attribute called `grad_fn`, which refers to the mathematical operator that create the variable. If `requires_grad` is set to False, `grad_fn` would be None. 

In our example where, `d = f(w_3b , w_4c)`, d's grad function would be the addition operator, since f adds it's to input together. Notice, addition operator is also the node in our graph that output's d. If our Tensor is a leaf node (initialised by the user), then the `grad_fn` is also None.

In [7]:
import torch 

a = torch.randn((3,3), requires_grad = True)

w1 = torch.randn((3,3), requires_grad = True)
w2 = torch.randn((3,3), requires_grad = True)
w3 = torch.randn((3,3), requires_grad = True)
w4 = torch.randn((3,3), requires_grad = True)

b = w1*a 
c = w2*a

d = w3*b + w4*c 

L = 10 - d

print("The grad fn for a is", a.grad_fn)
print("The grad fn for d is", d.grad_fn)


The grad fn for a is None
The grad fn for d is <AddBackward0 object at 0x1062e4eb8>


## Function

All mathematical operations in PyTorch are implemented by the `torch.nn.Autograd.Function` class. This class has two important member functions we need to look at. 
1. The first is it's `forward`  function, which simply computes the output using it's inputs. 
2. The `backward` function takes the incoming gradient coming from the the part of the network in front of it. The gradient to be backpropagated from a function f is basically the gradient that is backpropagated to f from the layers in front of it multiplied by the local gradient of the output of f with respect to it's inputs. This is exactly what the `backward`
function does. 

Let's again understand with our example of  $$ d = f(w_3b , w_4c) $$

1. `d` is our Tensor here. It's grad_fn is ``<ThAddBackward>``. This is basically the addition operation since the function that creates `d` adds inputs.
2. The `forward` function of the it's `grad_fn` receives the inputs $w_3b$ and $w_4c$ and adds them. This value is basically stored in the `d`
3. The `backward` function of the `<ThAddBackward>` basically takes the the incoming gradient from the further layers as the input. This is basically $\frac{\partial{L}}{\partial{d}}$ coming along the edge leading from `L` to `d`. This gradient is also the gradient of `L` w.r.t to `d` and is stored in `grad` attribute of the `d`. It can be accessed by calling `d.grad`.
4. It then takes computes the local gradients  $\frac{\partial{d}}{\partial{w_4c}}$ and $\frac{\partial{d}}{\partial{w_3b}}$.
5. Then the `backward` function multiplies the incoming gradient with the locally computed gradients respectively and "sends" the gradients to it's inputs by invoking the `backward` method of the `grad_fn` of their inputs. 
6. For example, the backward function of `<ThAddBackward>` associated with `d` invokes `backward` function of the `grad_fn` of the $w_4*c$ (Here, $w_4*c$ is a intermediate Tensor, and it's `grad_fn` is `<ThMulBackward>`.  At time of invocation of the `backward` function, the gradient $\frac{\partial{L}}{\partial{d}} * \frac{\partial{d}}{\partial{w_4c}} $ is passed as the input. 
7. Now, for the variable $w_4*c$, $\frac{\partial{L}}{\partial{d}} * \frac{\partial{d}}{\partial{w_4c}} $ becomes the incoming gradient, like $\frac{\partial{L}}{\partial{d}} $ was for $d$ in step 3 and the process repeats. 

Flow of gradients is demonstrated below. 

![Backpropagated Gradients](images/full_graph.png)

Algorithmically, here's how backpropagation happens with a computation graph. (Not the actual implementation, only representative)

In [8]:
# def backward (incoming_gradients):
# 	self.Tensor.grad = incoming_gradients

# 	for inp in self.inputs:
# 		if inp.grad_fn is not None:
# 			new_incoming_gradients = //
# 			  incoming_gradient * local_grad(self.Tensor, inp)
			
# 			inp.grad_fn.backward(new_incoming_gradients)
# 		else:
# 			pass


In order to compute derivatives in our neural network, we generally call `backward` on the Tensor representing our loss. Then, we backtrack through the graph starting from node representing the `grad_fn` of our loss.

As described above, the `backward` function is recursively called through out the graph as we backtrack. Once, we reach a leaf node, since the `grad_fn` is None, but stop backtracking through that path.

One thing to note here is that PyTorch gives an error if you call `backward` on vector-valued Tensor. This means you can only call `backward` on a scalar valued Tensor. In our example, if we assume a to be a vector valued Tensor, and call backward on L, it will throw up an error. 

In [9]:
import torch 

a = torch.randn((3,3), requires_grad = True)

w1 = torch.randn((3,3), requires_grad = True)
w2 = torch.randn((3,3), requires_grad = True)
w3 = torch.randn((3,3), requires_grad = True)
w4 = torch.randn((3,3), requires_grad = True)

b = w1*a 
c = w2*a

d = w3*b + w4*c 

L = (10 - d)

L.backward()

RuntimeError: grad can be implicitly created only for scalar outputs

There are two ways to overcome this.

1. If you just make a small change in the above code setting L to be the sum of all the errors, our problem will be solved

2. Second way is, for some reason have to absolutely call backward on a vector function, you can pass a torch.ones of size of shape of the tensor you are trying to call backward with. 

In [10]:
import torch 

a = torch.randn((3,3), requires_grad = True)

w1 = torch.randn((3,3), requires_grad = True)
w2 = torch.randn((3,3), requires_grad = True)
w3 = torch.randn((3,3), requires_grad = True)
w4 = torch.randn((3,3), requires_grad = True)

b = w1*a 
c = w2*a

d = w3*b + w4*c 

# Replace L = (10 - d) by 
L = (10 -d).sum()

L.backward()


In [11]:
import torch 

a = torch.randn((3,3), requires_grad = True)

w1 = torch.randn((3,3), requires_grad = True)
w2 = torch.randn((3,3), requires_grad = True)
w3 = torch.randn((3,3), requires_grad = True)
w4 = torch.randn((3,3), requires_grad = True)

b = w1*a 
c = w2*a

d = w3*b + w4*c 

# Replace L = (10 - d) by 
L = (10 -d)

L.backward(torch.ones(L.shape))


In this way, we can have gradients for every Tensor , and we can update them using Optimisation algorithm of our choice. 

In [12]:
learning_rate = 0.5
w1 = w1 - learning_rate * w1.grad
